In [1]:
import pandas as pd
import numpy

def isNaN(string):
    return string != string

gender_dialog_rd = pd.read_csv("C:/Users/Admin/24[7]_Project/processed_text/combined_lines_chars.csv", names=["charid","movieid","charname_x","lines","gender"],header=1, index_col=False)

#print(type(gender_dialog_rd))

listofrows = gender_dialog_rd.to_numpy().tolist()

listofrows = listofrows[0:20000]

print(len(listofrows))

for li in listofrows:
    if( (li[4] not in ['m','f']) or (isNaN(li[4]))):
        listofrows.remove(li)
        #Symbols and characters removal here
        #print(len(li[3]),li)
        
print("After removal: ",len(listofrows))

20000
After removal:  16320


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english')) 


In [3]:
def remove_punct(strin):
    punct_str = '!"#$%&\'()*+,-./:;?@[\]^_`{|}~ '
    restr = ""
    for letter in strin:
        if(letter not in punct_str):
            restr = restr+letter
    return restr

#from nltk.corpus import wordnet

#def similarity(word1, word2):
#    s=0.0
#    if wordFromList1 and wordFromList2: #Thanks to @alexis' note
 #           s = wordFromList1[0].wup_similarity(wordFromList2[0])
 #   return s

In [4]:
ds_for_cnn = []

for rows in listofrows:
    dialogue_tokens = word_tokenize(str(rows[3]))
    new_wrds_arr=[]
    for words in dialogue_tokens:
        if ( (len(remove_punct(words)) > 2) and (words.lower() not in stop_words)):
            new_wrds_arr.append(remove_punct(words.lower()))
        
    #print(new_arr)
    ds_for_cnn.append([new_wrds_arr,rows[4]])
    
print(len(ds_for_cnn))

16320


In [5]:
for items in ds_for_cnn:
    if((isNaN(items[1])) or (items[1]=='?')):
        ds_for_cnn.remove(items)
        

In [6]:
new_arr = []
for items in ds_for_cnn:
    if(len(items[0])>10):
        new_arr.append(items)
        
print("after removing")

#new_arr
        

after removing


In [7]:
sentence_arr = []
class_arr = []
for items in new_arr:
    sentence_arr.append(items[0])
    if(items[1]=='m'):
        class_arr.append(0)
    else:
        class_arr.append(1)

In [21]:
from gensim.models import Word2Vec
 
model = Word2Vec(sentence_arr, min_count=1, size=100)
vocab = model.wv.vocab
wordsInVocab = len(vocab)


import numpy as np
 
def sent_vectorizer(sent, model):
    sent_vec = np.zeros(100)
    numw = 0
    for w in sent:
        try:
            sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
    return sent_vec / np.sqrt(sent_vec.dot(sent_vec))
 
V=[]

for sentence in sentence_arr:
    V.append(sent_vectorizer(sentence, model))


from numpy import dot
from numpy.linalg import norm

weight_matrix = [[0 for i in range(100)] for j in range(len(V))] 
 
#for i in range (len(V) - 1):
 #   for j in range(0, 100):
  #      weight_matrix[i][j] = V[i][j]
          # weight_matrix[i][j] = dot(V[i],V[j])/norm(V[i])/norm(V[j])
indx=0
for vitems in V:
    indy=0
    for vvitems in vitems:
        weight_matrix[indx][indy] = vvitems
        indy+=1
    indx+=1

print("Weight matrix loaded")

C:\Users\Admin\anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


Weight matrix loaded


In [30]:
sentence_arr

[['yeah',
  'got',
  'sears',
  'catalog',
  'thing',
  'going',
  'tube',
  'sock',
  'gig',
  'gon',
  'huge',
  'queen',
  'harry',
  'next',
  'week'],
 ['that',
  'swore',
  'never',
  'anything',
  'everyone',
  'else',
  'since',
  'except',
  'bogey',
  'party',
  'stunning',
  'gastrointestinal',
  'display'],
 ['like',
  'choice',
  'besides',
  'let',
  'enemy',
  'think',
  'orchestrating',
  'battle',
  'position',
  'power',
  'let',
  'pretend',
  'calling',
  'shots',
  'busy',
  'setting',
  'plan',
  'time',
  'woo',
  'bianca'],
 ['serious',
  'man',
  'whacked',
  'sold',
  'liver',
  'black',
  'market',
  'could',
  'buy',
  'new',
  'speakers'],
 ['joey',
  'dorsey',
  'permashitgrin',
  'wish',
  'could',
  'say',
  'moron',
  'number',
  'twelve',
  'class',
  'model',
  'mostly',
  'regional',
  'stuff',
  'rumored',
  'big',
  'tube',
  'sock',
  'coming'],
 ['bred',
  'mothers',
  'liked',
  'guys',
  'like',
  'that',
  'grandmothers',
  'gene',
  'pool',
 

In [22]:
import tensorflow as tf

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, GlobalMaxPooling1D, Embedding

max_features = 5000
maxlen = 100
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

sent_train = sequence.pad_sequences(V, maxlen=maxlen)


cnn_model = Sequential()
cnn_model.add(Embedding(max_features,embedding_dims,input_length=maxlen))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(250,3,padding='valid',activation='relu',strides=1))
cnn_model.add(GlobalMaxPooling1D())

weight_matrx_test = weight_matrix[-100:]
class_arr_test = class_arr[-100:]

weight_mtrx_train = weight_matrix[0:-100]
class_arr_train = class_arr[0:-100]

cnn_model.add(Flatten())
cnn_model.add(Dense(hidden_dims))

cnn_model.add(Dense(1))
cnn_model.add(Activation('sigmoid'))


cnn_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn_model.fit(weight_mtrx_train, class_arr_train, batch_size=32,epochs=20,validation_data=(weight_matrx_test, class_arr_test))


Train on 1303 samples, validate on 100 samples
Epoch 1/20
1303/1303 [==============================] - 2s 2ms/sample - loss: 0.5378 - accuracy: 0.7767 - val_loss: 0.5370 - val_accuracy: 0.8000
Epoch 2/20
1303/1303 [==============================] - 1s 939us/sample - loss: 0.5243 - accuracy: 0.7874 - val_loss: 0.5407 - val_accuracy: 0.8000
Epoch 3/20
1303/1303 [==============================] - 1s 967us/sample - loss: 0.5235 - accuracy: 0.7874 - val_loss: 0.5138 - val_accuracy: 0.8000
Epoch 4/20
1303/1303 [==============================] - 1s 943us/sample - loss: 0.5190 - accuracy: 0.7874 - val_loss: 0.5114 - val_accuracy: 0.8000
Epoch 5/20
1303/1303 [==============================] - 1s 948us/sample - loss: 0.5203 - accuracy: 0.7874 - val_loss: 0.5270 - val_accuracy: 0.8000
Epoch 6/20
1303/1303 [==============================] - 1s 960us/sample - loss: 0.5218 - accuracy: 0.7874 - val_loss: 0.5135 - val_accuracy: 0.8000
Epoch 7/20
1303/1303 [==============================] - 1s 971us/sa

In [23]:
cnn_model.save("cnn_gender_classification_mew.model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: cnn_gender_classification_mew.model\assets


In [24]:
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           250000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 250)           37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)               62750     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                